In [91]:
!pip install python-dotenv

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Found existing installation: pandas 2.2.3
Uninstalling pandas-2.2.3:
  Successfully uninstalled pandas-2.2.3


ERROR: Too many arguments


^C


In [4]:
import os
import requests
from dotenv import load_dotenv
import json
import pandas as pd

load_dotenv()

True

# API Calls

In [5]:
API_KEY = os.environ.get("API_KEY")

headers = {
        "accept": "application/json",
        "Authorization": f"Bearer {API_KEY}"
    }

def get_movie():
    url = "https://api.themoviedb.org/3/movie/12?language=en-US"
    response = requests.get(url, headers=headers)

    return response.text

def get_genres():
    url = "https://api.themoviedb.org/3/genre/movie/list?language=en"
    response = requests.get(url, headers=headers)

    return response.text

def get_series_genres():
    url = "https://api.themoviedb.org/3/genre/tv/list?language=en"
    response = requests.get(url, headers=headers)

    return response.text

def popular_movies(page_num):
    url = f"https://api.themoviedb.org/3/movie/popular?language=en&page={page_num}"
    response = requests.get(url, headers=headers)

    return response.text

def popular_series(page_num):
    url = f"https://api.themoviedb.org/3/tv/popular?language=en-US&page={page_num}"
    response = requests.get(url, headers=headers)

    return response.text


def get_movie_by_genres(page_num, genres):
    url = f"https://api.themoviedb.org/3/discover/movie?include_adult=false&include_video=false&language=en-US&page={page_num}&sort_by=popularity.desc&with_genres={genres}"
    response = requests.get(url, headers=headers)

    return response.text

# Get Movies

In [6]:
json_string = get_genres()
data = json.loads(json_string)

genres_dict = {genre["id"]: genre["name"] for genre in data["genres"]}

genres_dict

{28: 'Action',
 12: 'Adventure',
 16: 'Animation',
 35: 'Comedy',
 80: 'Crime',
 99: 'Documentary',
 18: 'Drama',
 10751: 'Family',
 14: 'Fantasy',
 36: 'History',
 27: 'Horror',
 10402: 'Music',
 9648: 'Mystery',
 10749: 'Romance',
 878: 'Science Fiction',
 10770: 'TV Movie',
 53: 'Thriller',
 10752: 'War',
 37: 'Western'}

In [7]:
movies_csv_file = pd.DataFrame(columns=['Name','overview', 'genres', 'language', "popularity"]) 
movies_csv_file

,Name,overview,genres,language,popularity


In [13]:
genres_dict.keys()
genre_ids_list = list(genres_dict)

pages = 501
for idx, genre_id in enumerate(genre_ids_list):
    for num in range(1, pages): 
        try:     
            movies = get_movie_by_genres(page_num=num, genres=genre_id)
            movies_json = json.loads(movies)
            
            for movie in movies_json["results"]:
                genres = ", ".join([genres_dict.get(genre_id, "Unknown") for genre_id in movie["genre_ids"]])

                movie_data = pd.DataFrame([{
                    'Name': movie["original_title"],
                    'overview': movie["overview"],
                    'genres': genres,
                    'language': movie["original_language"],
                    'popularity': movie["popularity"]
                }])
                
                if not ((movies_csv_file['Name'] == movie_data.iloc[0]['Name']) & (movies_csv_file['overview'] == movie_data.iloc[0]['overview'])).any():
                    # If the movie doesn't exist, append the data
                    movies_csv_file = pd.concat([movies_csv_file, movie_data], ignore_index=True)

            print(f"----- Page {num} are added. -----")
        except Exception as e:
            print("----- Data is unavailable -----", e)

movies_csv_file.head()
movies_csv_file.to_csv('movies_genres.csv', index=False)
print("movies_genres.csv file has been created")

C:\Users\Asus\AppData\Local\Temp/ipykernel_23824/1072221689.py:24: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  movies_csv_file = pd.concat([movies_csv_file, movie_data], ignore_index=True)


----- Page 1 are added. -----
----- Page 2 are added. -----
----- Page 3 are added. -----
----- Page 4 are added. -----
----- Page 5 are added. -----
----- Page 6 are added. -----
----- Page 7 are added. -----
----- Page 8 are added. -----
----- Page 9 are added. -----
----- Page 10 are added. -----
----- Page 11 are added. -----
----- Page 12 are added. -----
----- Page 13 are added. -----
----- Page 14 are added. -----
----- Page 15 are added. -----
----- Page 16 are added. -----
----- Page 17 are added. -----
----- Page 18 are added. -----
----- Page 19 are added. -----
----- Page 20 are added. -----
----- Page 21 are added. -----
----- Page 22 are added. -----
----- Page 23 are added. -----
----- Page 24 are added. -----
----- Page 25 are added. -----
----- Page 26 are added. -----
----- Page 27 are added. -----
----- Page 28 are added. -----
----- Page 29 are added. -----
----- Page 30 are added. -----
----- Page 31 are added. -----
----- Page 32 are added. -----
----- Page 33 are

# Get Series

In [8]:
series_json_string = get_series_genres()
series_data = json.loads(series_json_string)

series_genres_dict = {genre["id"]: genre["name"] for genre in series_data["genres"]}
series_genres_dict

{10759: 'Action & Adventure',
 16: 'Animation',
 35: 'Comedy',
 80: 'Crime',
 99: 'Documentary',
 18: 'Drama',
 10751: 'Family',
 10762: 'Kids',
 9648: 'Mystery',
 10763: 'News',
 10764: 'Reality',
 10765: 'Sci-Fi & Fantasy',
 10766: 'Soap',
 10767: 'Talk',
 10768: 'War & Politics',
 37: 'Western'}

In [11]:
series_genres_dict.keys()
series_genre_ids_list = list(series_genres_dict)

pages = 501
for idx, genre_id in enumerate(series_genre_ids_list):
    print(series_genres_dict[genre_id])

Action & Adventure
Animation
Comedy
Crime
Documentary
Drama
Family
Kids
Mystery
News
Reality
Sci-Fi & Fantasy
Soap
Talk
War & Politics
Western


In [38]:
series_genres_dict.keys()
series_genre_ids_list = list(series_genres_dict)

pages = 501
for idx, genre_id in enumerate(series_genre_ids_list):
    for num in range(1, pages): 
        try:     
            movies = get_movie_by_genres(page_num=num, genres=genre_id)
            movies_json = json.loads(movies)
            
            for movie in movies_json["results"]:
                genres = ", ".join([genres_dict.get(genre_id, "Unknown") for genre_id in movie["genre_ids"]])

                movie_data = pd.DataFrame([{
                    'Name': movie["original_title"],
                    'overview': movie["overview"],
                    'genres': genres,
                    'language': movie["original_language"],
                    'popularity': movie["popularity"]
                }])
                
                if not ((movies_csv_file['Name'] == movie_data.iloc[0]['Name']) & (movies_csv_file['overview'] == movie_data.iloc[0]['overview'])).any():
                    # If the movie doesn't exist, append the data
                    movies_csv_file = pd.concat([movies_csv_file, movie_data], ignore_index=True)

            print(f"----- Page {num} are added. -----")
        except Exception as e:
            print("----- Data is unavailable -----", e)

movies_csv_file.head()
movies_csv_file.to_csv('movies_genres.csv', index=False)
print("movies_genres.csv file has been created")

C:\Users\Asus\AppData\Local\Temp/ipykernel_1412/2483175652.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  series_csv_file = pd.concat([series_csv_file, series_data], ignore_index=True)


----- Page 1 movies are added. -----
----- Page 2 movies are added. -----
----- Page 3 movies are added. -----
----- Page 4 movies are added. -----
----- Page 5 movies are added. -----
----- Page 6 movies are added. -----
----- Page 7 movies are added. -----
----- Page 8 movies are added. -----
----- Page 9 movies are added. -----
----- Page 10 movies are added. -----
----- Page 11 movies are added. -----
----- Page 12 movies are added. -----
----- Page 13 movies are added. -----
----- Page 14 movies are added. -----
----- Page 15 movies are added. -----
----- Page 16 movies are added. -----
----- Page 17 movies are added. -----
----- Page 18 movies are added. -----
----- Page 19 movies are added. -----
----- Page 20 movies are added. -----
----- Page 21 movies are added. -----
----- Page 22 movies are added. -----
----- Page 23 movies are added. -----
----- Page 24 movies are added. -----
----- Page 25 movies are added. -----
----- Page 26 movies are added. -----
----- Page 27 movies 

ConnectionError: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))